In [3]:
import jax
import jax.tree_util as jtu
import jax.numpy as jnp
import functools
from dataclasses import dataclass

import genjax
from genjax.typing import Callable, Any

console = genjax.pretty()


def emits_cc_gen_fn(v):
    @genjax.gen
    @genjax.dynamic_closure(v)
    def model(x):
        x = genjax.normal(jnp.sum(x), 1.0) @ "x"
        return x

    return model


@genjax.gen
def model():
    x = jnp.ones(5)
    gen_fn = emits_cc_gen_fn(x)
    v = gen_fn() @ "v"
    return (v, gen_fn)

In [4]:
key = jax.random.PRNGKey(314159)
key, tr = jax.jit(model.simulate)(key, ())
tr

BuiltinTrace
├── gen_fn
│   └── BuiltinGenerativeFunction
│       └── source
│           └── <function model>
├── args
│   └── tuple
├── retval
│   └── tuple
│       ├──  f32[]
│       └── BuiltinGenerativeFunction
│           └── source
│               └── DynamicClosure
│                   ├── fn
│                   │   └── <function model>
│                   └── dyn_args
│                       └── tuple
│                           └──  f32[5]
├── choices
│   └── Trie
│       └── :v
│           └── BuiltinTrace
│               ├── gen_fn
│               │   └── BuiltinGenerativeFunction
│               │       └── source
│               │           └── DynamicClosure
│               │               ├── fn
│               │               │   └── <function model>
│               │               └── dyn_args
│               │                   └── tuple
│               │                       └──  f32[5]
│               ├── args
│               │   └── tuple
│               ├── retval
│               │   └──  f32[]
│               ├── choices
│               │   └── Trie
│               │       └── :x
│               │           └── DistributionTrace
│               │               ├── gen_fn
│               │               │   └── Normal
│               │               ├── args
│               │               │   └── tuple
│               │               │       ├──  f32[]
│               │               │       └──  f32[]
│               │               ├── value
│               │               │   └──  f32[]
│               │               └── score
│               │                   └──  f32[]
│               ├── cache
│               │   └── Trie
│               └── score
│                   └──  f32[]
├── cache
│   └── Trie
└── score
    └──  f32[]